In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# from currency_converter import CurrencyConverter
# c = CurrencyConverter()
# Syntax: c.convert(1, 'GBP', 'USD')

from forex_python.converter import CurrencyRates
c = CurrencyRates()

In [3]:
date = '2022-10-22'
fp = f'../data/tagui/gc_tickers_{date}.csv'
data = pd.read_csv(fp
                 # ,dtype=str
                 ,encoding='ISO-8859-1' # to prevent unicode error
                 ,keep_default_na=True
                 ,dayfirst=True
                 ,infer_datetime_format=True
                 # ,parse_dates = date_cols
                  )
print(len(data))

data['VOLUME2'] = data['VOLUME'].str.replace(',', '', regex=True)
data['VOLUME2'] = data['VOLUME2'].str.replace('.', '', regex=True)
data['VOLUME2'] = data['VOLUME2'].str.replace(' million', '0000', regex=True)
data['VOLUME2'] = data['VOLUME2'].astype(int)

data = data.sort_values(by='VOLUME2', ascending=False)
print(data.dtypes)

data.head()

40
TICKER         object
NAME           object
INDUSTRY       object
EXCHANGE       object
SMA200PRICE    object
PRICE          object
VOLUME         object
VOLUME2         int64
dtype: object


,TICKER,NAME,INDUSTRY,EXCHANGE,SMA200PRICE,PRICE,VOLUME,VOLUME2
20,PEGY,Pineapple Energy,Oils/Energy,NYSE,$2.93,$5.16,15.06 million,15060000
21,PSX,Phillips 66,Oils/Energy,NYSE,$89.39,$100.44,3.24 million,3240000
8,FNB,F.N.B.,Finance,NYSE,$11.85,$13.83,3.22 million,3220000
9,ALL,Allstate,Finance,NYSE,$127.85,$120.08,3.19 million,3190000
22,MPLX,Mplx,Oils/Energy,NYSE,$31.69,$33.32,2.17 million,2170000


In [4]:
tmp = data.copy()
tmp = tmp.drop_duplicates(subset=['TICKER'])
tmp = tmp[tmp.TICKER.str.isupper()]
len(tmp)

31

In [5]:
tmp['PRICE2'] = tmp['PRICE'].apply(lambda x:x.replace('$', ''))
tmp['PRICE2'] = tmp['PRICE2'].apply(lambda x:x.replace('GBP', ''))
tmp['PRICE2'] = tmp['PRICE2'].apply(lambda x:x.replace('€', ''))
tmp['PRICE2'] = tmp['PRICE2'].apply(lambda x:x.replace(',', ''))
tmp['PRICE2'] = tmp['PRICE2'].astype(float)

tmp['SMA200PRICE2'] = tmp['SMA200PRICE'].apply(lambda x:x.replace('$', ''))
tmp['SMA200PRICE2'] = tmp['SMA200PRICE2'].apply(lambda x:x.replace('GBX ', ''))
tmp['SMA200PRICE2'] = tmp['SMA200PRICE2'].apply(lambda x:x.replace('€', ''))
tmp['SMA200PRICE2'] = tmp['SMA200PRICE2'].apply(lambda x:x.replace(',', ''))
tmp['SMA200PRICE2'] = tmp['SMA200PRICE2'].astype(float)

tmp.sample(5)


,TICKER,NAME,INDUSTRY,EXCHANGE,SMA200PRICE,PRICE,VOLUME,VOLUME2,PRICE2,SMA200PRICE2
18,ASND,Ascendis Pharma A/S,Medical,NYSE,$95.79,$108.50,"137,174",137174,108.50,95.79
10,FFBC,First Financial Bancorp.,Finance,NYSE,$21.37,$23.71,"382,820",382820,23.71,21.37
33,FIF,Finsbury Food Group,Utilities,GB,GBX 73.03,$85.80,"115,919",115919,85.80,73.03
1,VLCN,Volcon,Auto/Tires/Trucks,NYSE,$1.74,$1.63,"71,854",71854,1.63,1.74
19,RPHM,Reneo Pharmaceuticals,Medical,NYSE,$2.84,$2.97,"52,486",52486,2.97,2.84


In [6]:
def convert_to_usd(value, exchange):
    if exchange == 'GB':
        x = value * 0.01 # 1 GBX = 0.01000 GBP
        return c.convert('GBP', 'USD', x)
    return c.convert('EUR', 'USD', value)

In [7]:
# tmp['PRICE3'] = tmp.PRICE2.apply(lambda x:convert_to_usd(x, tmp.EXCHANGE) if )

tmp['PRICE_FINAL_USD'] = (tmp[['PRICE2', 'EXCHANGE']].apply(lambda x : convert_to_usd(x.PRICE2, x.EXCHANGE)
                                              if any([k in x.EXCHANGE for k in ['GB', 'EU']])
                                              else x.PRICE2, axis=1))

tmp['SMA200PRICE_FINAL_USD'] = (tmp[['SMA200PRICE2', 'EXCHANGE']].apply(lambda x : convert_to_usd(x.SMA200PRICE2, x.EXCHANGE)
                                              if any([k in x.EXCHANGE for k in ['GB', 'EU']])
                                              else x.SMA200PRICE2, axis=1))


tmp.sample(5)

,TICKER,NAME,INDUSTRY,EXCHANGE,SMA200PRICE,PRICE,VOLUME,VOLUME2,PRICE2,SMA200PRICE2,PRICE_FINAL_USD,SMA200PRICE_FINAL_USD
10,FFBC,First Financial Bancorp.,Finance,NYSE,$21.37,$23.71,"382,820",382820,23.71,21.37,23.710000,21.370000
8,FNB,F.N.B.,Finance,NYSE,$11.85,$13.83,3.22 million,3220000,13.83,11.85,13.830000,11.850000
18,ASND,Ascendis Pharma A/S,Medical,NYSE,$95.79,$108.50,"137,174",137174,108.50,95.79,108.500000,95.790000
19,RPHM,Reneo Pharmaceuticals,Medical,NYSE,$2.84,$2.97,"52,486",52486,2.97,2.84,2.970000,2.840000
33,FIF,Finsbury Food Group,Utilities,GB,GBX 73.03,$85.80,"115,919",115919,85.80,73.03,0.951616,0.809983


In [8]:
df = tmp[['TICKER', 'NAME', 'INDUSTRY', 'EXCHANGE', 'VOLUME2', 'PRICE_FINAL_USD','SMA200PRICE_FINAL_USD']].copy()
df.columns = ['TICKER', 'NAME', 'INDUSTRY', 'EXCHANGE', 'VOLUME', 'PRICE_USD','SMA200PRICE_USD']
df.head()

,TICKER,NAME,INDUSTRY,EXCHANGE,VOLUME,PRICE,SMA200PRICE
20,PEGY,Pineapple Energy,Oils/Energy,NYSE,15060000,5.16,2.93
21,PSX,Phillips 66,Oils/Energy,NYSE,3240000,100.44,89.39
8,FNB,F.N.B.,Finance,NYSE,3220000,13.83,11.85
9,ALL,Allstate,Finance,NYSE,3190000,120.08,127.85
22,MPLX,Mplx,Oils/Energy,NYSE,2170000,33.32,31.69


In [9]:
df['BUY'] = (df[['PRICE', 'SMA200PRICE']].apply(lambda x : 1
                                              if x.PRICE > x.SMA200PRICE
                                              else 0, axis=1))
df.head()

,TICKER,NAME,INDUSTRY,EXCHANGE,VOLUME,PRICE,SMA200PRICE,BUY
20,PEGY,Pineapple Energy,Oils/Energy,NYSE,15060000,5.16,2.93,1
21,PSX,Phillips 66,Oils/Energy,NYSE,3240000,100.44,89.39,1
8,FNB,F.N.B.,Finance,NYSE,3220000,13.83,11.85,1
9,ALL,Allstate,Finance,NYSE,3190000,120.08,127.85,0
22,MPLX,Mplx,Oils/Energy,NYSE,2170000,33.32,31.69,1


In [10]:
"""Percent Price Oscillator (PPO) 
 subtract the 200-day EMA from the 1-day EMA (closing price). 
This difference is then divided by the 200-day EMA, 
which shows the difference as a percentage of the 200-day EMA.
"""

df = (df.assign(**{'PPO': np.where(df.BUY == 1,
                                   np.round((df.PRICE - df.SMA200PRICE) / df.SMA200PRICE * 100, 2), 
                                   'N/A')}))

df['PPO'] = df['PPO'].astype(float)

# exclude  PPO's that are too large
# TODO make a flag column instead of removing them
df = df[df.PPO <= 100.0]

df.head()

,TICKER,NAME,INDUSTRY,EXCHANGE,VOLUME,PRICE,SMA200PRICE,BUY,PPO
20,PEGY,Pineapple Energy,Oils/Energy,NYSE,15060000,5.16,2.93,1,76.11
21,PSX,Phillips 66,Oils/Energy,NYSE,3240000,100.44,89.39,1,12.36
8,FNB,F.N.B.,Finance,NYSE,3220000,13.83,11.85,1,16.71
9,ALL,Allstate,Finance,NYSE,3190000,120.08,127.85,0,N/A
22,MPLX,Mplx,Oils/Energy,NYSE,2170000,33.32,31.69,1,5.14


In [11]:
df_final = df.copy()
df_final = df_final[df_final.BUY==1] # TODO use the PPO flag column instead (need to create first see above)

22
['PEGY', 'PSX', 'FNB', 'MPLX', 'DRI', 'WFRD', 'IBRX', 'PRVB', 'FFBC', 'STGW', 'LBAI', 'CVSG', 'QNST', 'ASND', 'FIF', 'RPHM', 'NBPE', 'NRC', 'ART', 'ULBI', 'AC', 'CFBK']


In [12]:

print(len(df_final))
print(list(df_final.TICKER))
df_final

,TICKER,NAME,INDUSTRY,EXCHANGE,VOLUME,PRICE,SMA200PRICE,BUY,PPO
20,PEGY,Pineapple Energy,Oils/Energy,NYSE,15060000,5.160000,2.930000,1,76.11
21,PSX,Phillips 66,Oils/Energy,NYSE,3240000,100.440000,89.390000,1,12.36
8,FNB,F.N.B.,Finance,NYSE,3220000,13.830000,11.850000,1,16.71
22,MPLX,Mplx,Oils/Energy,NYSE,2170000,33.320000,31.690000,1,5.14
24,DRI,Darden Restaurants,Retail/Wholesale,NYSE,1610000,133.320000,125.350000,1,6.36
23,WFRD,Weatherford International,Oils/Energy,NYSE,1350000,36.350000,28.910000,1,25.74
14,IBRX,ImmunityBio,Medical,NYSE,1230000,5.310000,4.350000,1,22.07
15,PRVB,Provention Bio,Medical,NYSE,894514,6.770000,4.790000,1,41.34
10,FFBC,First Financial Bancorp.,Finance,NYSE,382820,23.710000,21.370000,1,10.95
3,STGW,Stagwell,Business Services,NYSE,356397,7.540000,6.810000,1,10.72
